In [1]:
##Librerias
from bs4 import BeautifulSoup
import requests
import pandas as pd
import psycopg2


#####################################
###Conexion####
conn = psycopg2.connect(host="localhost", database="webscraping", user="postgres", password="1234", port="5432")
conn.autocommit = True 
cur = conn.cursor()

##Query
cur.execute('''CREATE TABLE Noticias_Scraping(N_Id int NOT NULL,\
Titulo text,\
Descripcion text,\
Autor varchar(50),Fuente varchar(50), Fecha varchar(50));''')
conn.commit()

i=1
count = 2
tablelist=[]
url='https://www.bbc.com/mundo/topics/c2lej05epw5t'

while count <=50 :
    req = requests.get(url)
    soup = BeautifulSoup(req.text, 'html.parser')
    league = soup.find('ol',class_ = 'gs-u-m0 gs-u-p0 lx-stream__feed qa-stream')
    for team in league.find_all('li', class_= 'lx-stream__post-container'):
        num = team.find('a', class_ = 'qa-heading-link lx-stream-post__header-link').text.strip()
        t = team.find('p', class_ = 'lx-stream-related-story--summary qa-story-summary')
        if t is not None:
            nombre= t.text.strip()
        else:
            nombre =None
        t2 = team.find('p', class_ = 'qa-contributor-name lx-stream-post__contributor-name gel-long-primer gs-u-m0')
        if t2 is not None:
            Tipo = t2.text.strip()
        else:
            Tipo = None
        t3 = team.find('p', class_ = 'qa-contributor-role lx-stream-post__contributor-description gel-brevier gs-u-m0')
        if t3 is not None:
            fuente=t3.text.strip()
        else :
            fuente = None
        t4 = team.find('span', class_ = 'gs-u-vh qa-visually-hidden-meta')
        if t4 is not None:
            fecha=t4.text.strip()
        else:
            fecha = None
        cur.execute("INSERT INTO Noticias_Scraping VALUES (%s,%s,%s,%s,%s,%s)",(i,num,nombre,Tipo,fuente,fecha))
        json ={
            "Titulo": num,
            "Descripcion": nombre,
            "Autor": Tipo,
            "Fuente": fuente,
            "Fecha": fecha
        }
        tablelist.append(json)
        i+=1
    subS = 'https://www.bbc.com/mundo/topics/c2lej05epw5t/page/'+ str(count)
    url = subS
    count+=1

df= pd.DataFrame(tablelist)
df.to_csv('Noticias.csv')
   
conn.close()

In [2]:
import psycopg2
import pymongo
from pymongo import MongoClient
from pymongo.errors import ConnectionFailure

cliente = MongoClient('localhost',27017)
db = cliente["Noticias"]
Noticias = db.noticias

conn = psycopg2.connect(host="localhost", database="webscraping", user="postgres", password="1234", port="5432")
cur = conn.cursor()
Query = "Select * from noticias_scraping "
cur.execute(Query)
scraping= cur.fetchall()

for row in scraping:
    teamingLeague = [{
        "Id" : row[0],
        "Titulo" : row [1],
        "Descripcion" : row[2],
        "Autor" : row[3],
        "Fuente" : row[4],
        "Fecha" : row[5]
    }]
    Noticias.insert_many(teamingLeague)
conn.close()